# Formatting

---

For each datapoint we want the following attributes:

- [x] Johnson county
- [x] weekend
- [ ] night
- [x] 'business hours' (M-F, 8-5:30)
- [ ] Campus building
- [x] Year of school

In [1]:
import pandas as pd
import geopandas as gpd

/home/user/.miniconda3/envs/maps/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/user/.miniconda3/envs/maps/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Johnson County Coords
coords_f = '../data/johnson-cnty-coords.csv'
coords_df = pd.read_csv(coords_f)

coords_df.head()

,timestamp,latitude,longitude
0,2016-07-27 15:35:24,41.683614,-91.504861
1,2017-03-17 17:24:27,41.683686,-91.503358
2,2016-07-27 15:37:24,41.683698,-91.504880
3,2016-07-27 15:38:25,41.683710,-91.504871
4,2016-07-27 15:28:39.467000,41.683757,-91.504225


### Parse Timestamps into CST timezone from UTC

In [3]:
def parse_df_times(df):
    # Copy as to not mutate OG data
    copy_df = df.copy()
    # Parse Google UTC timestamp
    copy_df['parsed_timestamp'] = pd.to_datetime(copy_df['timestamp'], utc=True, infer_datetime_format=True)
    # Convert to Central Time
    copy_df['cst_timestamp'] = copy_df.parsed_timestamp.dt.tz_convert('America/Chicago')
    
    # Remove the verbose columns and rename to match
    slim_df = copy_df[['cst_timestamp', 'latitude', 'longitude']]
    slim_df.rename(columns={'cst_timestamp': 'timestamp'}, inplace=True)
    
    return slim_df

time_parsed_df = parse_df_times(coords_df)

/home/user/.miniconda3/envs/maps/lib/python3.5/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


### Time Manipulation

Add attrs:
- hour of day (nights)
- day of week (weekend, 'business_hrs')
- date (year of school)

In [10]:
time_parsed_df.head()

,timestamp,latitude,longitude
0,2016-07-27 10:35:24-05:00,41.683614,-91.504861
1,2017-03-17 12:24:27-05:00,41.683686,-91.503358
2,2016-07-27 10:37:24-05:00,41.683698,-91.504880
3,2016-07-27 10:38:25-05:00,41.683710,-91.504871
4,2016-07-27 10:28:39.467000-05:00,41.683757,-91.504225


In [4]:
test = time_parsed_df.iloc[0].timestamp

test.weekday_name # wed
test.hour # 10
test.strftime('%x') # '07/26/2016'

/home/user/.miniconda3/envs/maps/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  This is separate from the ipykernel package so we can avoid doing imports until


'07/27/16'

In [51]:
from datetime import datetime
import pytz

cst = pytz.timezone('America/Chicago')
years = {
    'freshman': [datetime(year=2013, month=8, day=1, tzinfo=cst), datetime(year=2014, month=5, day=15, tzinfo=cst)],
    'sophomore': [datetime(year=2014, month=6, day=16, tzinfo=cst), datetime(year=2015, month=5, day=15, tzinfo=cst)],
    'junior': [datetime(year=2015, month=5, day=16, tzinfo=cst), datetime(year=2016, month=5, day=15, tzinfo=cst)],
    'senior': [datetime(year=2016, month=5, day=16, tzinfo=cst), datetime(year=2017, month=5, day=18, tzinfo=cst)],
    'postgrad': [datetime(year=2017, month=5, day=19, tzinfo=cst), datetime(year=2018, month=10, day=1, tzinfo=cst)]
}

def assign_year(timestamp):
    for yr_obj in years.items():
        yr = yr_obj[0]
        start = yr_obj[1][0]
        end = yr_obj[1][1]
        
        if ((timestamp > start) and (timestamp < end)):
            return yr
    return 'none'

def is_weekend(day):
    return ((day == 'Saturday') or (day == 'Sunday'))

def is_business(timestamp):
    day = timestamp.weekday_name
    hour = timestamp.hour
    business_day = is_weekend(day) == False
    business_hrs = ((hour >= 7) and (hour < 19))
    return (business_day and business_hrs)

def format_df_cols(df):
    # Non-mutable
    copy_df = df.copy()
    
    # Add time attributes
    copy_df['weekday'] = df.timestamp.dt.weekday_name
    copy_df['dayhour'] = df.timestamp.dt.hour
    
    # Dimensions
    copy_df['weekend'] = copy_df['weekday'].apply(is_weekend)
    copy_df['year'] = copy_df['timestamp'].apply(assign_year)
    copy_df['business'] = copy_df['timestamp'].apply(is_business)
    
    return copy_df

df_attrs = format_df_cols(time_parsed_df)

/home/user/.miniconda3/envs/maps/lib/python3.5/site-packages/ipykernel_launcher.py:27: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead


In [52]:
df_attrs.head()

,timestamp,latitude,longitude,weekday,dayhour,weekend,year,business
0,2016-07-27 10:35:24-05:00,41.683614,-91.504861,Wednesday,10,False,senior,True
1,2017-03-17 12:24:27-05:00,41.683686,-91.503358,Friday,12,False,senior,True
2,2016-07-27 10:37:24-05:00,41.683698,-91.504880,Wednesday,10,False,senior,True
3,2016-07-27 10:38:25-05:00,41.683710,-91.504871,Wednesday,10,False,senior,True
4,2016-07-27 10:28:39.467000-05:00,41.683757,-91.504225,Wednesday,10,False,senior,True


In [53]:
df_attrs.business.value_counts()

False    343463
True     228419
Name: business, dtype: int64

In [54]:
df_attrs.year.value_counts()

postgrad    245404
senior      215624
junior      106221
none          4633
Name: year, dtype: int64

## Campus Buildings